# ChatGPT Prompting

En esta unidad veremos como utilizar la libreria de OpenAI para realizar consultas a ChatGPT y poder utilizar sus respuestas para luego crear un programa mas complejo.

Esta unidad es un resumen traducido con algunas modificaciones del [curso de Andrew Ng de Deeplearning.ai](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

In [ ]:
!pip install openai --upgrade --quiet

In [ ]:
from openai import OpenAI

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [ ]:
#@title [Opcional] cargar el token desde colab para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import userdata
    api_key = userdata.get('OPENAI_TOKEN')

In [ ]:
# defaults to os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
def generar_texto(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # aleatoriedad de las respuestas del modelo
    )
    return response.choices[0].message.content

In [ ]:
generar_texto("Hola")

In [ ]:
generar_texto("En que año estamos?")

In [ ]:
generar_texto("Mi nombre es Hernan")

In [ ]:
generar_texto("Cual es mi nombre?")

## Como escribir un prompt:
- Principio 1: Escribir instrucciones claras y precisas
- Principio 2: Separar un problema grande en pequeños problemas simples

## Principio 1: Escribir instrucciones claras y precisas

### Tacticas

#### Tactica 1: Usar delimitadores para diferenciar claramente partes distintas de la entrada
- Los delimitadores pueden ser cualquier caracter o secuencia de caracteres como: ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
Debes expresar lo que quieres que un modelo haga \
proporcionando instrucciones que sean lo más claras \
y específicas posibles. Esto guiará al modelo hacia la \
salida deseada y reducirá las posibilidades de recibir \
respuestas irrelevantes o incorrectas. No confundas escribir \
una instrucción clara con escribir una instrucción breve. \
En muchos casos, las instrucciones más largas proporcionan más \
claridad y contexto para el modelo, lo que puede resultar en \
salidas más detalladas y relevantes.
"""

prompt = f"""
Resume el texto delimitado por tres acentos invertidos \
en una sola frase.
```{text}```
"""
response = generar_texto(prompt)
print(response)

#### Tactica 2: Pedir una salida estructurada
- JSON, HTML

In [ ]:
prompt = f"""
Genera una lista de tres títulos de libros inventados junto \
con sus autores y géneros.
Proporciónalos en formato JSON con las siguientes claves:
book_id, title, author, genre.
"""
response = generar_texto(prompt)
print(response)

#### Ejercicio 1:

Hacer zero shot de detección de spam utilizando las tacticas antes mencionadas y utilizando el conocimiento masivo que tiene comprimido chatgpt en su arquitectura de mas de 100 billones de parametros.

Para ello pedirle al modelo que reconozca si un texto es spam o que y especificar cual debe ser el formato de la respuesta. Utilizar la tactica de delimitar el texto que queremos analizar.

#### Tactica 3: Pedir al modelo que verifique que se cumplan ciertas condiciones

In [ ]:
text_1 = f"""
¡Hacer una taza de té es fácil! Primero, necesitas hervir \
un poco de agua. Mientras eso ocurre, toma una taza y coloca \
una bolsita de té en ella. Una vez que el agua esté lo \
suficientemente caliente, simplemente viértela sobre la bolsita \
de té. Déjalo reposar un rato para que el té pueda infusionarse. \
Después de unos minutos, retira la bolsita de té. Si lo deseas, \
puedes añadir azúcar o leche al gusto. ¡Y eso es todo! Tienes \
para ti una deliciosa taza de té para disfrutar.
"""
prompt = f"""
Se te proporcionará un texto delimitado por comillas triples.
Si contiene una secuencia de instrucciones,
reescribe esas instrucciones en el siguiente formato:

Paso 1 - ...
Paso 2 - ...
…
Paso N - ...

Si el texto no contiene una secuencia de instrucciones,
simplemente escribe: \"No se proporcionaron pasos.\"

\"\"\"{text_1}\"\"\"
"""
response = generar_texto(prompt)
print("Respuesta para el Texto 1:")
print(response)

In [ ]:
text_2 = f"""
El sol brilla con fuerza hoy, y los pájaros están cantando. \
Es un hermoso día para dar un paseo por el parque. Las flores \
están floreciendo y los árboles se mecen suavemente con la brisa. \
La gente está fuera disfrutando del agradable clima. Algunos \
están haciendo picnics, mientras que otros están jugando juegos \
o simplemente relajándose en el césped. Es un día perfecto para \
pasar tiempo al aire libre y apreciar la belleza de la naturaleza.
"""
prompt = f"""
Se te proporcionará un texto delimitado por comillas triples.
Si contiene una secuencia de instrucciones,
reformula esas instrucciones en el siguiente formato:

Paso 1 - ...
Paso 2 - ...
…
Paso N - ...

Si el texto no contiene una secuencia de instrucciones,
simplemente escribe: \"No se proporcionaron pasos.\"

\"\"\"{text_2}\"\"\"
"""
response = generar_texto(prompt)
print("Respuesta para el Texto 2:")
print(response)

#### Tactica 4: "Few-shot" prompting
- Mostrar pocos ejemplos para que el modelo sepa como responder

In [ ]:
prompt = f"""
Tu tarea es responder con un estilo consistente.

<niño>: Enséñame sobre la paciencia.

<abuelo>: El río que talla el valle más profundo fluye \
de una fuente modesta; la sinfonía más grandiosa se origina \
de una sola nota; el tapiz más intrincado comienza con un solo hilo.

<niño>: Enséñame sobre la resiliencia.
"""
response = generar_texto(prompt)
print(response)

#### Ejercicio 2:

Hacer few shot clasification de tweets que son graciosos o no segun nuestro criterio. 

Cuando utilizar esto y cuando finetunning de un modelo mas pequeño?

## Principio 2: Separar un problema grande en pequeños problemas simples

#### Tactica 1: Definir claramente los pasos requeridos para completar una tarea

In [ ]:
text = f"""
En un encantador pueblo, los hermanos Jack y Jill emprendieron \
una misión para buscar agua de un pozo en lo alto de una colina.\
Mientras ascendían, cantando con alegría, la desgracia ocurrió: \
Jack tropezó con una piedra y cayó rodando colina abajo, con Jill \
siguiéndole el paso. Aunque algo magullados, la pareja regresó a \
casa a abrazos reconfortantes. A pesar del percance, su espíritu \
aventurero permaneció inquebrantable y continuaron explorando con \
deleite.
"""
# ejemplo 1
prompt_1 = f"""
Realiza las siguientes acciones:
1 - Resumir el texto delimitado por tres acentos invertidos \
en una sola frase.
2 - Traducir el resumen al francés.
3 - Enumera cada nombre en el resumen en francés.
4 - Genera un objeto json que contenga las siguientes claves: \
resumen_en_frances, num_nombres.

Separa tus respuestas con saltos de línea.

Texto:
```{text}```
"""
response = generar_texto(prompt_1)
print("Respuesta al prompt 1:")
print(response)

#### Solicitar que la salida tenga un formato especifico

In [ ]:
prompt_2 = f"""
Tu tarea es realizar las siguientes acciones:
1 - Resumir el siguiente texto delimitado por
<> en una sola frase.
2 - Traducir el resumen al francés.
3 - Enumera cada nombre en el resumen en francés.
4 - Genera un objeto json que contenga las siguientes
claves: resumen_en_frances, num_nombres.

Utiliza el siguiente formato:
Texto: <texto a resumir>
Resumen: <resumen>
Traducción: <traducción del resumen>
Nombres: <lista de nombres en el resumen francés>
Salida JSON: <json con resumen y num_nombres>

Texto: <{text}>
"""
response = generar_texto(prompt_2)
print("Respuesta para el prompt 2:")
print(response)

#### Tactica 2: Instruye al modelo para que elabore su propia solución antes de apresurarse a una conclusión.

In [ ]:
prompt = f"""
Determina si la solución del estudiante es correcta o no.

Pregunta:
Estoy construyendo una instalación de energía solar y necesito \
ayuda para calcular los aspectos financieros.

El terreno cuesta $100 por pie cuadrado
Puedo comprar paneles solares por $250 por pie cuadrado
Negocié un contrato de mantenimiento que me costará una tarifa \
plana de $100k al año, y un adicional de $10 por pie cuadrado.
¿Cuál es el costo total para el primer año de operaciones en función \
del número de pies cuadrados?
Solución del estudiante:
Dejemos x como el tamaño de la instalación en pies cuadrados.
Costos:

Costo del terreno: 100x
Costo del panel solar: 250x
Costo de mantenimiento: 100,000 + 10x
Costo total: 100x + 250x + 100,000 + 10x = 450x + 100,000
"""
response = generar_texto(prompt)
print(response)

#### Notar que la solucion del alumno no es correcta.
#### Podemos solucionar esto instruyendo al modelo que trabaje la solucion de la siguiente manera:

In [ ]:
prompt = f"""
Tu tarea es determinar si la solución del estudiante es correcta o no.
Para resolver el problema haz lo siguiente:

Primero, resuelve el problema por ti mismo.
Luego compara tu solución con la del estudiante y evalúa si la \
solución del estudiante es correcta o no.
No decidas si la solución del estudiante es correcta hasta que \
hayas resuelto el problema tú mismo.
Usa el siguiente formato:
Pregunta:
```
pregunta aqui
```
Solucion del Estudiante:
```
solucion del estudiante aqui
```
Solucion correcta:
```
pasos para resolver y tu solución aquí
```
¿La solución del estudiante es la misma que la solución real \
recién calculada?
```
si o no
```
Calificación del estudiante:
```
correcto or incorrecto
```

Pregunta:
```
Estoy construyendo una instalación de energía solar y necesito \
ayuda para calcular los aspectos financieros.
- El terreno cuesta $100 por pie cuadrado
- Puedo comprar paneles solares por $250 por pie cuadrado
- Negocié un contrato de mantenimiento que me costará una tarifa \
plana de $100k al año, y un adicional de $10 por pie cuadrado
¿Cuál es el costo total para el primer año de operaciones en función \
del número de pies cuadrados?
``` 
Solución del estudiante:
```
Dejemos x como el tamaño de la instalación en pies cuadrados.
Costos:
1. Costo del terreno: 100x
2. Costo del panel solar: 250x
3. Costo de mantenimiento: 100,000 + 100x
Costo total: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Solucion correcta:
"""
response = generar_texto(prompt)
print(response)

## Limitaciones del Modelo: alucionaciones

Puede ser que el modelo responda cosas con seguridad pero que no tengan ningun sentido

#### Nota sobre la barra invertida
- Utilizamos la barra invertida `\` para hacer que el texto entre en la pantalla sin insertar una nueva linea '\n'.
- GPT-3 no es afectado por los caracteres de nueva linea.  Pero cuando trabajamos con LLMs en general, hay que 

## Referencias:
- [Prompt engineering from OpenAI](https://platform.openai.com/docs/guides/prompt-engineering)
- [Best practices for prompt engineering with the OpenAI API](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-the-openai-api)
- [Prompt engineering from Meta for Llama3](https://llama.meta.com/docs/how-to-guides/prompting/)
- [Libreria de OpenAI para python](https://github.com/openai/openai-python)
- [Curso de Prompt Engineering para Programadores](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)